In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "vscode"

from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from sklearn.preprocessing import MinMaxScaler


import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
database = pd.read_csv(r"C:\Users\Anson\OneDrive\Anson File\PythonApplication_anson\3_Kaggle\4_20210419_StockPrice_Prediction_LSTM\NVDA.csv", sep=',')
database['Date/Time']=pd.to_datetime(database['Date/Time'])
database.set_axis(database['Date/Time'], inplace=True)

stock_data_original=database[["Open","High","Low","Volume","Close","Close"]]
stock_data_values=stock_data_original.values


#IMPORTANT
# print('database[Date/Time] \n\n',stock_data_original)
# print('datastock_data \n\n',stock_data_values)
# print('stock_data.iloc[:,4]',stock_data_original.iloc[:,1:5])
# print('stock_data.iloc[:,5]',stock_data_original.iloc[:,5])

In [3]:
split_percent = 0.80
split = int(split_percent*len(stock_data_original))

In [4]:
# Divide the data for training and testing
# 80% for training data, 20% for testing data

stock_data_X = stock_data_values[:,0:stock_data_values.shape[1]-1]
stock_data_y = stock_data_values[:,stock_data_values.shape[1]-1]


# Normalization for stock_data for neural network

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
stock_data_X_scaled = scaler_X.fit_transform(stock_data_X)
stock_data_y_scaled = scaler_y.fit_transform(stock_data_y.reshape(-1,1)).reshape(-1)


stock_data_train_X = stock_data_X_scaled[:split]
stock_data_train_y = stock_data_y_scaled[:split]

stock_data_test_X = stock_data_X_scaled[split:]
stock_data_test_y = stock_data_y_scaled[split:]


# Split for datatime 

date_train = database['Date/Time'][:split]
date_test = database['Date/Time'][split:]


In [5]:
#look back day is 50
look_back = 50

train_generator = TimeseriesGenerator(data=stock_data_train_X, targets=stock_data_train_y, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(data=stock_data_test_X, targets=stock_data_test_y, length=look_back, batch_size=1)



In [6]:
Keras_model = Sequential()
Keras_model.add(LSTM(100, activation='relu',input_shape=(look_back,stock_data_train_X.shape[1])))
Keras_model.add(Dropout(0.3))
Keras_model.add(Dense(1))
Keras_model.compile(optimizer='adam',loss='mean_squared_error')

num_epochs = 10
Keras_model.fit_generator(train_generator, epochs=num_epochs, verbose=1)



C:\Users\Anson\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1240: UserWarning:

`model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.

Epoch 1/10
143/143 [==============================] - 36s 231ms/step - batch: 71.0000 - size: 19.9860 - loss: 0.0017
Epoch 2/10
143/143 [==============================] - 31s 220ms/step - batch: 71.0000 - size: 19.9860 - loss: 3.0066e-04
Epoch 3/10
143/143 [==============================] - 34s 237ms/step - batch: 71.0000 - size: 19.9860 - loss: 4.8155e-04
Epoch 4/10
143/143 [==============================] - 39s 271ms/step - batch: 71.0000 - size: 19.9860 - loss: 2.0970e-04
Epoch 5/10
143/143 [==============================] - 34s 239ms/step - batch: 71.0000 - size: 19.9860 - loss: 2.1254e-04
Epoch 6/10
143/143 [==============================] - 37s 258ms/step - batch: 71.0000 - size: 19.9860 - loss: 1.9777e-04
Epoch 7/10
143/143 [====================

In [7]:
prediction = Keras_model.predict_generator(test_generator)


# stock_data_train = stock_data_train.reshape(-1)
# stock_data_test = stock_data_test.reshape(-1)
# prediction = prediction.reshape(-1)
# stock_data = close_data.reshape((-1))


C:\Users\Anson\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1301: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.

C:\Users\Anson\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning:

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.



In [8]:
def predict(num_prediction, Keras_model):
    prediction_list_X = stock_data_test_X[-look_back:]
    prediction_list_y = stock_data_test_y[-look_back:]
    

#     print('prediction_list_X first' ,prediction_list_X)
#     print('prediction_list_y first' ,prediction_list_y)
    
    for _ in range(num_prediction):
        x = prediction_list_X[-look_back:]
        x = x.reshape((1, look_back, stock_data_test_X.shape[1]))
        out = Keras_model.predict(x)[0][0]
        prediction_list_y = np.append(prediction_list_y, out)
#         print('prediction_list_y',prediction_list_y)
        
        
    prediction_list_y = prediction_list_y[look_back-1:]

    print('prediction_list_y',prediction_list_y)

    return prediction_list_y

In [9]:
def predict_dates(num_prediction):

    last_date = database['Date/Time'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    
    print('prediction_dates',prediction_dates)
    return prediction_dates


In [10]:
# Prediction the Future Price

num_prediction = 1
forecast = predict(num_prediction, Keras_model)
forecast_dates = predict_dates(num_prediction)

# print('forecast',forecast)
# print('forecast_dates',forecast_dates)

prediction_list_y [1.        0.8072952]
prediction_dates [Timestamp('2021-06-03 00:00:00', freq='D'), Timestamp('2021-06-04 00:00:00', freq='D')]


In [11]:

# print('stock_data_train_y.reshape',stock_data_train_y)
# print('stock_data_train_y.reshape(-1,1)',stock_data_train_y.reshape(-1,1))
# print('stock_data_test_y',stock_data_test_y)

stock_data_train_y = scaler_y.inverse_transform(stock_data_train_y.reshape(-1,1))
stock_data_train = stock_data_train_y.reshape(-1)

# print('stock_data_test_y \n\n',stock_data_test_y)
stock_data_test_y = scaler_y.inverse_transform(stock_data_test_y.reshape(-1,1))
stock_data_test = stock_data_test_y.reshape(-1)

forecast = scaler_y.inverse_transform(forecast.reshape(-1,1))
forecast = forecast.reshape(-1)


##IMPORTANT
# print('stock_data_train',stock_data_train)
# print('stock_data_test',stock_data_test)
# print('forecast',forecast)

# print('date_train',date_train)
# print('date_test',date_test)
# print('forecast_dates',forecast_dates)


In [12]:
# Graph for the stock prediction

trace1 = go.Scatter(x = date_train, y = stock_data_train, mode = 'lines', name = 'Data')
trace2 = go.Scatter(x = date_test,y = stock_data_test,mode='lines', name = 'Ground Truth')
trace3 = go.Scatter(x = forecast_dates,y = forecast, mode = 'lines', name = 'Future')
layout = go.Layout(title = "Nvidia Stock", xaxis = {'title' : "Date"}, yaxis = {'title' : "Close"})
fig = go.Figure(data=[trace1,trace2,trace3], layout=layout)
fig.show()